# Generalized polynomial based work

This notebook covers comparing two datasets, one of which will be considered the "base" dataset, and one will be the dataset which we compare to.
The environment is the normal cvasl environment (mrilander).

### import needed libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)

import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(0, '../') # path to functions
from cvasl import file_handler as fh # 
from cvasl import mold #
from cvasl import carve
from cvasl import seperated
from cvasl.file_handler import Config

### get data

In [ ]:
# set up data pull
config = Config()
root_mri_directory = config.get_directory('raw_data')

## Setting the base and comapared datasets
In this example we will use the TOP dataset as our base, and mriStroke as the other dataset.
This is highly problematic for anything stratified by gender, but we will overlook that for now,
as both datasets have about 50% women.

In [ ]:
base = os.path.join(root_mri_directory, 'assembled/top_stitched.csv')
compared = os.path.join(root_mri_directory, 'assembled/StrokeMRI_stitched.csv')
# in the future the below should be base_data and the tricks will skip
our_top_data = pd.read_csv(base)
dataframe_compared = pd.read_csv(compared)

In [ ]:
our_top_data.describe()

We may have a mistake in our_top_data, white matter hyperintensities...also the total flows have outlier max values. Let's look

### temporary trick to deal with data inconsistency

In [ ]:
our_top_data['GM_vol'] = our_top_data['GM_vol_Liter']
our_top_data['WM_vol'] = our_top_data['WM_vol_Liter']
our_top_data['CSF_vol'] = our_top_data['CSF_vol_Liter']
our_top_data['GM_ICVRatio'] = our_top_data['GM_ICVRatio_ratio GM/ICV'] 
our_top_data['WMH_vol'] = our_top_data['GMWM_ICVRatio_ratio (GM+WM)/ICV']
our_top_data['WMH_count'] = our_top_data['WMH_count_n lesions (integer)']
our_top_data['DeepWM_B'] = our_top_data['DeepWM_B_SD/mean']
our_top_data['DeepWM_L'] = our_top_data['DeepWM_L_SD/mean']
our_top_data['DeepWM_L'] = our_top_data['DeepWM_R_SD/mean']
our_top_data['ACA_B']= our_top_data['ACA_B_SD/mean']    
our_top_data['ACA_L']= our_top_data['ACA_L_SD/mean']           
our_top_data['ACA_R']= our_top_data['ACA_R_SD/mean']            
our_top_data['MCA_B']= our_top_data['MCA_B_SD/mean']      

In [ ]:
plt.scatter(our_top_data['Age'],our_top_data['WMH_count'])

In [ ]:
our_top_data[our_top_data['WMH_count'] > 100]

In [ ]:
our_top_data[our_top_data['WMH_count'] > 100]['Age']

# Needs scientist decision
Someone with 570 WMH at age of 41 is abnormal, is there a mistake in the data? Also note the volumes are not particularly large.Or a sick patient?
Should we  drop such outliers? Automatically?

In [ ]:
plt.scatter(dataframe_compared['Age'],dataframe_compared['WMH_count'])

## Moving on...

In [ ]:
# now we find common columns; this will be easier when all is same formatted
shared_columns = (
        dataframe_compared.columns.intersection(our_top_data.columns)).to_list()

In [ ]:
shared_columns

### create base polynomials

In [ ]:
# find common columns

In [ ]:
special_data_column = 'Age'

In [ ]:
other_columns = [
    'GM_vol',
     'WM_vol',
     'CSF_vol',
     'GM_ICVRatio',
     'WMH_vol',
     'WMH_count',
     'DeepWM_B',
     'DeepWM_L',
     'ACA_B',
     'ACA_L',
     'ACA_R',
     'MCA_B', ]

In [ ]:
our_top_data

In [ ]:
# below functions must go into main library

In [ ]:


def polyfit_second_degree_to_df(
        dataframe,
        special_column_name,
        other_column_names,
):
    """
    This function creates a polynomial for two columns.
    It returns the coefficients
    
    :param dataframe: dataframe variable
    :type dataframe: pandas.dataFrame
    :param special_column_name: string of column you want to graph against
    :type  special_column_name: str
    :param other_column_name: string of column you want to graph
    :type other_column_name: str
    :param degree_poly: either 1,2 or 3 only
    :type  degree_poly: int


    :returns: coeffiects
    :rtype: :class:`~numpy.ndarray`
    """
    list_as = []
    list_bs = []
    list_cs = []
    list_columns = []
    dataframe = dataframe.dropna()
    for interest_column_name in other_column_names:
        xscat = np.array(pd.to_numeric(dataframe[special_column_name]))
        yscat = np.array(pd.to_numeric(dataframe[interest_column_name]))
        coefficients = np.polyfit(xscat, yscat, 2 ) #2 = degree_poly
        list_columns.append(interest_column_name)
        list_as.append(coefficients[0])
        list_bs.append(coefficients[1])
        list_cs.append(coefficients[2])
    d = {'column':list_columns,'coefficient_a':list_as, 'coefficient_b':list_bs, 'coefficient_c':list_cs}
    coefficien_dataframe = pd.DataFrame(d)
   
    return coefficien_dataframe


In [ ]:
def derived_function(column, a, b, c):
    return a * (column**2) + b * column + c


In [ ]:
cos_dataframe = polyfit_second_degree_to_df(
        our_top_data,#dataframe_base,
        special_data_column,
        other_columns,
)
cos_dataframe

In [ ]:
projected_columns = []
coefficients = ['coefficient_a', 'coefficient_b', 'coefficient_c']
for column in our_top_data[shared_columns].columns:
    projected_columns.append(column + '_projected')
    row = cos_dataframe[cos_dataframe['column'] == column]
    if row.empty:
        # The columns that appear "weird" below (eg. `Series([], dtype: float64)`)
        # are the columns not found in `cos_dataframe`, so they don't have associated coefficients..
        print('skipping', column)
        continue
    a, b, c = row[coefficients].values.flatten().tolist()
    our_top_data[column + '_projected'] = derived_function(our_top_data['Age'], a, b, c)
our_top_data

In [ ]:
shared_columns_new =  ['GM_vol',
 'WM_vol',
 'CSF_vol',
 'GM_ICVRatio',
 'WMH_vol',
 'WMH_count',
 'DeepWM_B',
 'DeepWM_L',
 'ACA_B',
 'ACA_L',
 'ACA_R',
 'MCA_B',]

In [ ]:
difference_columns = []
for column in our_top_data[shared_columns_new].columns:
    difference_columns.append(column+ '_diff')
    our_top_data[column + '_diff'] = our_top_data[column] - our_top_data[column + '_projected']
    our_top_data[column + '_abs_diff'] = abs(our_top_data[column] - our_top_data[column + '_projected'])
our_top_data    

## Now we want to do the same to the compared dataframe

In [ ]:
#dataframe_compared

projected_columns = []
coefficients = ['coefficient_a', 'coefficient_b', 'coefficient_c']
for column in dataframe_compared[shared_columns].columns:
    projected_columns.append(column + '_projected')
    row = cos_dataframe[cos_dataframe['column'] == column]
    if row.empty:
        # The columns that appear "weird" below (eg. `Series([], dtype: float64)`)
        # are the columns not found in `cos_dataframe`, so they don't have associated coefficients..
        print('skipping', column)
        continue
    a, b, c = row[coefficients].values.flatten().tolist()
    dataframe_compared[column + '_projected'] = derived_function(dataframe_compared['Age'], a, b, c)
difference_columns = []
for column in dataframe_compared[shared_columns_new].columns:
    difference_columns.append(column+ '_diff')
    dataframe_compared[column + '_diff'] = dataframe_compared[column] - dataframe_compared[column + '_projected']
    dataframe_compared[column + '_abs_diff'] = abs(dataframe_compared[column] - dataframe_compared[column + '_projected'])
dataframe_compared

In [ ]:
shared_columns_rel = ['GM_vol',
 'WM_vol',
 'CSF_vol',
 'GM_ICVRatio',
 'WMH_vol',
 'WMH_count',
 'DeepWM_B',
 'DeepWM_L',
 'ACA_B',
 'ACA_L',
 'ACA_R',
 'MCA_B',]

In [ ]:
for column in our_top_data[shared_columns_rel].columns:
    plt.figure()
    plt.title('base_dataframe ' +column)
    plt.scatter(our_top_data['Age'],our_top_data[column])
    plt.scatter(our_top_data['Age'],our_top_data[column + '_projected'])

In [ ]:
for column in dataframe_compared[shared_columns_rel].columns:
    plt.figure()
    plt.title('compared ' +column)
    plt.scatter(dataframe_compared['Age'],dataframe_compared[column], color='purple')
    plt.scatter(dataframe_compared['Age'],dataframe_compared[column + '_projected'])

## describe the differences in base dataframe, the compared dataframe

In [ ]:
list_diff_dc =dataframe_compared.columns[dataframe_compared.columns.str.contains("diff")].to_list()
list_diff_top = our_top_data.columns[our_top_data.columns.str.contains("diff")].to_list()

In [ ]:
our_top_data[list_diff_top].describe()

In [ ]:
dataframe_compared[list_diff_dc].describe()

In [ ]:
dataframe_compared[list_diff_dc].describe().loc['max']

In [ ]:
our_top_data[list_diff_dc].describe().loc['max'] 

In [ ]:
# if this number is positive or zero we are golden!
outer_top_minus_outer_mri_top_poly = our_top_data[list_diff_dc].describe().loc['max'] - dataframe_compared[list_diff_dc].describe().loc['max'] 
outer_top_minus_outer_mri_top_poly

In [ ]:
# needs recode
(len(outer_top_minus_outer_mri_top_poly) - len(outer_top_minus_outer_mri_top_poly[outer_top_minus_outer_mri_top_poly > 0])) / len(outer_top_minus_outer_mri_top_poly)


So about a third of the values get into a range outside the expected...but by how much?

We need to look into (should be compared as a percentage of average values?):



WM_vol_abs_diff          -0.001561


CSF_vol_abs_diff         -0.031943


GM_ICVRatio_abs_diff     -0.021200


WMH_vol_abs_diff        -64.356226

In [ ]:
# hardd coding to be replaces
WM_vol_abs_diff = -0.001561
WM_vol_abs_diff /our_top_data['WM_vol'].mean()

In [ ]:
CSF_vol_abs_diff =  -0.031943
CSF_vol_abs_diff  / our_top_data['CSF_vol'].mean()

so we do see a 10% difference in csf volumne

In [ ]:
GM_ICVRatio_abs_diff = -0.021200
GM_ICVRatio_abs_diff /our_top_data['GM_ICVRatio'].mean()

and a 4% difference in GMICV radio

In [ ]:
WMH_vol_abs_diff  = -64.356226
WMH_vol_abs_diff/ our_top_data['WMH_vol'].mean()

but a huge difference in white matter pter intensity. As the true nature of the curve is shown only in the later ages?
But maybe step 2 is to apply a transformation matrix to the outliers,.


Then we are talking about an algorithm that identifies the outliers, and pushes them through a transformation matrix?

Anyways, first some histograms

In [ ]:
our_top_data[list_diff_top].columns

In [ ]:
our_top_data['WMH_count_diff'].max()

In [ ]:
# for each feature make a histogram for  difference
for column_d in our_top_data[list_diff_top].columns:
    if 'abs' not in column_d: # add if to get rid of abs
        plt.figure()
        plt.title(column_d + ' base distribution')
        our_top_data[column_d].hist()


In [ ]:
# for each feature make a histogram for  difference
for column_d in dataframe_compared[list_diff_top].columns:
    if 'abs' not in column_d: # add if to get rid of abs
        plt.figure()
        plt.title(column_d + ' compared distribution')
        dataframe_compared[column_d].hist(color='purple')

In [ ]:
# get ready to rescale values on histograms to match
multiplier = len(our_top_data)/len(dataframe_compared)
multiplier

In [ ]:
# for each feature make a histogram for  difference on both datasets
for column_d in our_top_data[list_diff_top].columns:
    if our_top_data[column_d].max() > dataframe_compared[column_d].max():
        find_true_max = our_top_data[column_d].max()
    else:
        find_true_max = dataframe_compared[column_d].max()
        
    if 'abs' not in column_d: # add if to get rid of abs

        base_df_histogram , bin_edges= np.histogram(our_top_data[column_d], bins=10, range=(-find_true_max , find_true_max ), density=None, weights=None)
        comapred_df_histogram , bin_edges = np.histogram(dataframe_compared[column_d], bins=10, range=(-find_true_max , find_true_max ), density=None, weights=None)
        scaled_comparison_histogram = comapred_df_histogram * multiplier
        plt.figure(figsize=[10,6])

        plt.bar(bin_edges[:10], base_df_histogram, width = 0.03, color='#0504aa',alpha=0.5)
        plt.xlim(min(bin_edges), max(bin_edges))
        plt.grid(axis='y', alpha=0.75)

        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.ylabel('Frequency',fontsize=15)
        #plt.title('Difference from Polynomial Distribution Histograms',fontsize=15)
        #plt.show()
        plt.bar(bin_edges[:10], scaled_comparison_histogram, width = 0.03, color='#FF00FF',alpha=0.5)
        plt.xlim(min(bin_edges), max(bin_edges))
        plt.grid(axis='y', alpha=0.75)
        plt.xlabel('Residuals Scaled',fontsize=15)
        plt.ylabel('Frequency',fontsize=15)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.ylabel('Frequency',fontsize=15)
        plt.title('Difference from Polynomial Distribution Histogram ' + column_d,fontsize=15)
        plt.show()

In [ ]:
## Now we need to describe the residual differences

# So we want our histogram to be comparable to other histograms...we can scale every histogram to a 100 patient population,not so coincidentally, 

In [ ]:
# scaled_histogram_difference = top_dif_histogram - scaled_mri_dif_histogram
# hundred_scaled_histo_diff = scaled_histogram_difference * (100 / len(dataframe_for_difference_top))

In [ ]:
# plt.figure(figsize=[10,6])

# plt.bar(bin_edges[:10], hundred_scaled_histo_diff, width = 0.03, color='#0504aa',alpha=0.5)
# plt.xlim(min(bin_edges), max(bin_edges))
# plt.grid(axis='y', alpha=0.75)
# plt.xlabel('Value Difference between two distributions',fontsize=15)
# #plt.ylabel('Frequency',fontsize=15)
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.ylabel('Frequency',fontsize=15)
# plt.title('"Residuals" from Polynomial Distribution Histograms both datasets',fontsize=15)
# #plt.show()